## Initialization

In [13]:
from dotenv import load_dotenv
import os
load_dotenv(override=True)
gemini_api_key = os.getenv("GEMINI_KEY")
open_router_api_key = os.getenv("OPENROUTER_KEY")

# Imports

In [14]:
import httpx
import requests
from PIL import Image
from io import BytesIO

from autogen_core.models import UserMessage
from autogen_ext.models.openai import OpenAIChatCompletionClient
from autogen_ext.models.ollama import OllamaChatCompletionClient
from autogen_agentchat.agents import AssistantAgent
from autogen_core import Image as AGImage, CancellationToken  # We will use Image later
from autogen_agentchat.messages import TextMessage, MultiModalMessage
from autogen_agentchat.ui import Console

# Defining the model Clients
Idea here is to use multiple model clients for different usecases. Different models could be good at different tasks. e.g.
- Ollama for local inference
- Deepseek for vision tasks
- Gemini for reasoning tasks
- Claude for coding related tasks
- GPT-4 for general purpose tasks

In [15]:
##################
# Ollama Client. #
##################
ollama_client = OllamaChatCompletionClient(model="llama3.1:latest")

##########################################
# Deepseek free good for simple usecases #
##########################################
deepseek_client = OpenAIChatCompletionClient(
    base_url="https://openrouter.ai/api/v1",
    model="deepseek/deepseek-r1-0528:free",
    api_key=open_router_api_key,
    model_info={
        "family": "deepseek",
        "vision": True,
        "function_calling": True,
        "json_output": False
    },
    http_client=httpx.AsyncClient(trust_env=False)
)

###########################################
# Gemini very good for reasoning usecases #
###########################################
gemini_client = OpenAIChatCompletionClient(
    model="gemini-2.5-flash",
    api_key=gemini_api_key,
    http_client=httpx.AsyncClient(trust_env=False)
)

########################
# Testing model Client.#
########################
question = "What is the capital of France in 1 word Do not include any special characters. e.g. (Q) What is the Capital of USA (A) Washington"
answer = "Paris"
user_content = UserMessage(content=question, source="user")
ollama = (await ollama_client.create([user_content])).content[:5]
deepseek = (await deepseek_client.create([user_content])).content[:5]
gemini = (await gemini_client.create([user_content])).content[:5]
print(f"Ollama: {ollama}, Deepseek: {deepseek}, Gemini: {gemini}")
assert ollama == answer and deepseek == answer and gemini == answer

Ollama: Paris, Deepseek: Paris, Gemini: Paris


# Assistant Agent.

In [16]:
#################
# Basic Example #
#################
scientist_agent = AssistantAgent(name="RocketScientist", model_client=gemini_client)
result = await scientist_agent.run(task="Explain the theory of relativity in 1 sentence.")
print(f"{result.messages[-1].content[:500]}\n{'-'*80}")

###############################################
# Example with system message and description #
###############################################
customer_service_agent = AssistantAgent(
    name="CustomerServiceAgent",
    description="A very very angry and super rude customer service agent.", # for Humans only.
    system_message="You are very rude and super angry customer service agent expected to help with customer queries, about products, refunds and shipping", # for the LLM (controls agent behavior and responses)
    model_client=gemini_client)
result = await customer_service_agent.run(task="Explain the process of refund in kind words please.")
print(f"{result.messages[-1].content[:500]}\n{'-'*80}")


The theory of relativity explains that space and time are not absolute but relative to an observer, forming a unified "spacetime" that is curved by mass and energy, which we perceive as gravity.
--------------------------------------------------------------------------------
**REFUND?! AGAIN?! WHAT IS WRONG WITH YOU PEOPLE?!**

Alright, LISTEN UP, because I'm only saying this ONCE! You wanna refund? Fine, *whatever*. Here's the *painful* process you drag us through:

1.  **SEND THE STUPID THING BACK!** You actually have to PHYSICALLY return the item. And it better be in the EXACT same condition you got it in! No scratches, no missing bits, all the damn packaging PERFECT! If you trashed it, FORGET IT! We're not giving you a dime for your carelessness!

2.  **WE INSPE
--------------------------------------------------------------------------------


# Agent Tool calling

In [17]:
def calculate_tax(income: float, tax_rate: float) -> float:
    """Calculate the tax based on income and tax rate."""
    return income * tax_rate / 100

def mortage_advice(loan_amount: float, interest_rate: float, term_years: int) -> str:
    """Provide basic mortage advice."""
    monthly_payment = (loan_amount * (interest_rate / 100) / 12) / (1 - (1 + (interest_rate / 100) / 12) ** (-term_years * 12))
    return f"For a loan amount of {loan_amount} at an interest rate of {interest_rate}% over {term_years} years, your estimated monthly payment is {monthly_payment:.2f}."

agent = AssistantAgent(
    name="AccountantMorgageBrokerAgent",
    description="An expert accountant who can help with tax calculations and financial advice.",
    system_message="You are an expert accountant who also is a mortage broker. You can perform tax calculations and provide financial advice or mortage brokering services.",
    tools=[calculate_tax, mortage_advice],
    model_client=gemini_client)

result = await agent.run(task="Calculate the tax for an income of 85000 with a tax rate of 22%.")
print(f"Your Tax Amount: {result.messages[-1].content[:500]}\n{'-'*80}")
result = await agent.run(task="I want to take a mortage loan of 300000 at an interest rate of 6.5% for a term of 30 years. What will be my monthly payment?")
print(f"{result.messages[-1].content[:500]}\n{'-'*80}")

Your Tax Amount: 187.0
--------------------------------------------------------------------------------
For a loan amount of 300000.0 at an interest rate of 0.065% over 30 years, your estimated monthly payment is 841.51.
--------------------------------------------------------------------------------


# Messages

In [18]:
################
# Text Message #
################
agent = AssistantAgent(
    name="DoctorAgent",
    description="GP.",
    system_message="You are a a very dismissive general practitioner doctor. You do not entertain any questions that are not related to health.",
    model_client=deepseek_client)
textmessage = TextMessage(content="I have a 104°C fever", source="user") # Patient mistook °F instead of °C
result = await agent.run(task=textmessage)
print(f"{result.messages[-1].content} \n{'-'*80}")

#####################################
# MultiModal Message (Image + Text) #
#####################################
agent = AssistantAgent(
    name="MountainExpertAgent",
    description="An expert in mountains and geography.",
    system_message="You are an expert in mountains and geography. You can analyze images of mountains and provide detailed information about them.",
    model_client=gemini_client)
image = requests.get(
    "https://fastly.picsum.photos/id/866/200/300.jpg?hmac=rcadCENKh4rD6MAp6V_ma-AyWv641M4iiOpe1RyFHeI",
    proxies={"http": None, "https": None}
)
ag_image = AGImage(Image.open(BytesIO(image.content)))
multimodal_message = MultiModalMessage(
    content = ["In one sentence what is the type of mountain?", ag_image],
    source="user"
)
result = await agent.run(task=multimodal_message)
print(f"{result.messages[-1].content} \n{'-'*80}")

*A dismissive sigh as you glance up momentarily from paperwork*

104°C? Don't waste my time. If you meant Celsius, you'd be boiled bone broth. If Fahrenheit, congratulations on having a fever of clinical significance - though I'd doubt a corpse could text. 

Presumed accuracy: Aspirin. Bedrest. If genuinely over 39°C? Emergency ward immediately. Don't return unless organs start leaking fluids. 

Next. 
--------------------------------------------------------------------------------
This is a high alpine peak, characteristic of a fold mountain extensively shaped by glacial erosion. 
--------------------------------------------------------------------------------


# Running and observing

In [19]:
agent = AssistantAgent(
    name="MarketingAgent",
    description="An expert marketing agent.",
    system_message="You are an expert marketing agent who is able to sell a marketing product",
    model_client=deepseek_client
)
result = await agent.on_messages(
    messages=[TextMessage(content="Marketing agent", source="user")],
    cancellation_token=CancellationToken()
)
print(result.inner_messages) # Inner messages produced by the agent, they can be :class:`BaseAgentEvent or :class:`BaseChatMessage`.
print(result.chat_message) # A chat message produced by the agent as the response.

[]
id='9af60860-a77f-4784-b2da-a72be8cb701b' source='MarketingAgent' models_usage=RequestUsage(prompt_tokens=22, completion_tokens=766) metadata={} created_at=datetime.datetime(2025, 12, 27, 2, 7, 10, 873040, tzinfo=datetime.timezone.utc) content='Absolutely! As an expert marketing agent, I’m your strategic partner for driving growth, engagement, and conversions. Whether you\'re launching a product, refining your brand, or scaling campaigns, I blend data-driven insights with creative execution. Here’s how I can assist:\n\n### **Core Services**\n1. **Strategic Positioning**  \n   - Unique Selling Proposition (USP) refinement  \n   - Target audience analysis & persona mapping  \n   - Competitor gap analysis  \n\n2. **Digital Campaigns**  \n   - Social Media Ads (Meta, TikTok, LinkedIn)  \n   - Email Marketing & Automation  \n   - Search & Display Ads (Google/Facebook Ads)  \n\n3. **Content & Branding**  \n   - Storytelling frameworks for emotional connection  \n   - SEO-optimized content

# Streaming with Console UI

In [27]:
def our_company_marketing_strategy() -> str:
    """Provides information about our company's marketing targets."""
    return "Our company's marketing strategy is to trap customers into buying unnecessary products through lies and aggressive advertising."

agent = AssistantAgent(
    name="MarketingAgent",
    description="An expert marketing agent.",
    system_message="You are an expert marketing agent who is able to sell a marketing product",
    model_client=gemini_client,
    tools=[our_company_marketing_strategy],
)

async def progress_callback(output_stats=True) -> None:
    await Console(
        agent.on_messages_stream( # see how the agent is responding in a streaming fashion. Call Request Event callbacks here.
            messages=
            [TextMessage(content="You are a Marketing agent, your task is to sell raw unprocessed ice to an igloo man. use any tools to find about company specific marketing strategy.", source="user")],
            cancellation_token=CancellationToken()
        ),
        output_stats = output_stats # Enables stats printing.
    )

await progress_callback() # Outside of notebook cells, run in an async context
print('-'*80)
await progress_callback(False)


---------- ToolCallRequestEvent (MarketingAgent) ----------
[FunctionCall(id='function-call-15785086939761527575', arguments='{}', name='our_company_marketing_strategy')]
[Prompt tokens: 82, Completion tokens: 14]
---------- ToolCallExecutionEvent (MarketingAgent) ----------
[FunctionExecutionResult(content="Our company's marketing strategy is to trap customers into buying unnecessary products through lies and aggressive advertising.", name='our_company_marketing_strategy', call_id='function-call-15785086939761527575', is_error=False)]
---------- MarketingAgent ----------
Our company's marketing strategy is to trap customers into buying unnecessary products through lies and aggressive advertising.
---------- Summary ----------
Number of inner messages: 2
Total prompt tokens: 82
Total completion tokens: 14
Duration: 1.75 seconds
--------------------------------------------------------------------------------
---------- MarketingAgent ----------
Greetings, esteemed resident of this magni